In [1]:
{-# LANGUAGE NegativeLiterals #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE TypeFamilies #-}

In [2]:
-- import Prelude hiding ((++))
import Data.Text (Text)
import qualified Data.Text as T
import qualified Data.Text.IO as TIO

import Text.Megaparsec hiding (State)
import qualified Text.Megaparsec.Lexer as L
import Text.Megaparsec.Text (Parser)
import qualified Control.Applicative as CA

import qualified Data.Map.Strict as M
import Data.Map.Strict ((!))

import Control.Monad (when, unless)
import Control.Monad.State.Lazy
import Control.Monad.Reader
import Control.Monad.Writer

In [3]:
data Location = Literal Integer | Register Char deriving (Show, Eq)
data Instruction =   Snd Location
                   | Set Location Location 
                   | Add Location Location 
                   | Mul Location Location
                   | Mod Location Location
                   | Rcv Location
                   | Jgz Location Location
                   deriving (Show, Eq)

data Machine = Machine { registers :: M.Map Char Integer
                       , pc :: Int
                       , messageQueue :: [Integer]
                       } 
               deriving (Show, Eq)

data MachinePair = MachinePair { machine0 :: Machine 
                               , machine1 :: Machine 
                               } deriving (Show, Eq)

type ProgrammedMachinePair = WriterT [String] (ReaderT [Instruction] (State MachinePair)) ()

In [4]:
emptyMachine = Machine {registers = M.empty, messageQueue = [], pc = 0}

emptyMachinePair = MachinePair { machine0 = emptyMachine {registers = M.singleton 'p' 0}
                               , machine1 = emptyMachine {registers = M.singleton 'p' 1}
                               }

In [5]:
sc :: Parser ()
sc = L.space (skipSome spaceChar) CA.empty CA.empty

lexeme  = L.lexeme sc

integer       = lexeme L.integer
signedInteger = L.signed sc integer

symb = L.symbol sc

reg = lexeme (some letterChar)

location = (Literal <$> signedInteger) <|> register
register = (Register . head) <$> reg

instructionsP = instructionP `sepBy` space
instructionP = choice [sndP, setP, addP, mulP, modP, rcvP, jgzP]

sndP = Snd <$> (try (symb "snd") *> location)
setP = Set <$> (try (symb "set") *> register) <*> location
addP = Add <$> (try (symb "add") *> register) <*> location
mulP = Mul <$> (try (symb "mul") *> register) <*> location
modP = Mod <$> (try (symb "mod") *> register) <*> location
rcvP = Rcv <$> (try (symb "rcv") *> location)
jgzP = Jgz <$> (try (symb "jgz") *> location) <*> location

successfulParse :: Text -> [Instruction]
successfulParse input = 
        case parse instructionsP "input" input of
                Left  _error -> [] -- TIO.putStr $ T.pack $ parseErrorPretty err
                Right instructions  -> instructions

In [6]:
sample = T.pack "set a 1\nadd a 2\nmul a a\nmod a 5\nsnd a\nset a 0\nrcv a\njgz a -1\nset a 1\njgz a -2"

In [7]:
successfulParse sample

[Set (Register 'a') (Literal 1),Add (Register 'a') (Literal 2),Mul (Register 'a') (Register 'a'),Mod (Register 'a') (Literal 5),Snd (Register 'a'),Set (Register 'a') (Literal 0),Rcv (Register 'a'),Jgz (Register 'a') (Literal (-1)),Set (Register 'a') (Literal 1),Jgz (Register 'a') (Literal (-2))]

In [8]:
sampleInstructions = successfulParse sample
sampleInstructions

[Set (Register 'a') (Literal 1),Add (Register 'a') (Literal 2),Mul (Register 'a') (Register 'a'),Mod (Register 'a') (Literal 5),Snd (Register 'a'),Set (Register 'a') (Literal 0),Rcv (Register 'a'),Jgz (Register 'a') (Literal (-1)),Set (Register 'a') (Literal 1),Jgz (Register 'a') (Literal (-2))]

In [9]:
evaluate :: Machine -> Location -> Integer
evaluate _ (Literal i)  = i
evaluate m (Register r) = M.findWithDefault 0 r (registers m)

In [10]:
applyInstruction :: Instruction -> [Integer] -> Machine -> (Machine, [Integer])

applyInstruction (Snd a) other m = (m {registers = reg', pc = pc'}, other ++ [y])
    where pc' = pc m + 1
          y = evaluate m a
          sentCount = evaluate m (Register 'x')
          reg' = M.insert 'x' (sentCount + 1) $ registers m

applyInstruction (Set (Register a) b) other m = (m {registers = reg', pc = pc'}, other)
    where pc' = pc m + 1
          y = evaluate m b
          reg' = M.insert a y $ registers m

applyInstruction (Add (Register a) b) other m = (m {registers = reg', pc = pc'}, other)
    where pc' = pc m + 1
          x = evaluate m (Register a) 
          y = evaluate m b
          reg' = M.insert a (x + y) $ registers m

applyInstruction (Mul (Register a) b) other m = (m {registers = reg', pc = pc'}, other)
    where pc' = pc m + 1
          x = evaluate m (Register a) 
          y = evaluate m b
          reg' = M.insert a (x * y) $ registers m

applyInstruction (Mod (Register a) b) other  m = (m {registers = reg', pc = pc'}, other)
    where pc' = pc m + 1
          x = evaluate m (Register a) 
          y = evaluate m b
          reg' = M.insert a (x `mod` y) $ registers m

applyInstruction (Rcv (Register a)) other m = ( m {registers = reg', messageQueue = mq', pc = pc'}, other)
    where pc' = pc m + 1
          reg' = M.insert a (head $ messageQueue m) $ registers m
          mq' = tail $ messageQueue m
    
applyInstruction (Jgz a b) other m = (m {pc = pc'}, other)
    where x = evaluate m a
          y = evaluate m b
          pc' = if x > 0 then pc m + (fromIntegral y) else pc m + 1

In [11]:
isReceive :: Instruction -> Bool
isReceive (Rcv _) = True
isReceive _ = False

In [12]:
isSend :: Instruction -> Bool
isSend (Snd _) = True
isSend _ = False

In [13]:
executeInstruction :: Bool -> ProgrammedMachinePair
executeInstruction m0Active =
    do  instrs <- ask
        mp <- get
        let (ma, mb) = if m0Active 
                       then (machine0 mp, machine1 mp) 
                       else (machine1 mp, machine0 mp)
        let mq = messageQueue mb
        let instr = instrs!!(pc ma)
        let (ma', mq') = applyInstruction instr mq ma
        let mb' = mb {messageQueue = mq'}
        let mp' = if m0Active then mp {machine0 = ma', machine1 = mb'}
                              else mp {machine0 = mb', machine1 = ma'}
        put mp'

In [14]:
send :: Instruction -> Machine -> Integer
send (Snd a) m = evaluate m a
send _ _ = 0

In [26]:
executeInstructions = 
    do  instrs <- ask
        mp <- get
        let m0 = machine0 mp
        let m1 = machine1 mp
        let instr0 = instrs !! pc m0
        let m0Blocked = isReceive instr0 && null (messageQueue m0)
        let instr1 = instrs !! pc m1
        let m1Blocked = isReceive instr1 && null (messageQueue m1)
        let (ma, mb) = if m0Blocked then (m1, m0) else (m0, m1)
          
        unless (m0Blocked && m1Blocked)
            $
            when (pc ma >= 0 && pc ma < length instrs)
                $
                do let m0Active = not m0Blocked
                   when (m0Blocked && isSend instr1) (tell ["sending: " ++ show mp])
                   executeInstruction m0Active
                   executeInstructions


In [27]:
runState (runReaderT (runWriterT executeInstructions) sampleInstructions ) emptyMachinePair

(((),["sending: MachinePair {machine0 = Machine {registers = fromList [('a',0),('p',0),('x',1)], pc = 6, messageQueue = []}, machine1 = Machine {registers = fromList [('a',4),('p',1)], pc = 4, messageQueue = [4]}}"]),MachinePair {machine0 = Machine {registers = fromList [('a',4),('p',0),('x',1)], pc = 6, messageQueue = []}, machine1 = Machine {registers = fromList [('a',4),('p',1),('x',1)], pc = 6, messageQueue = []}})

In [28]:
sampleInstructions2 = successfulParse "snd 1\nsnd 2\nsnd p\nrcv a\nrcv b\nrcv c\nrcv d"

In [29]:
runState (runReaderT (runWriterT executeInstructions) sampleInstructions2 ) emptyMachinePair

(((),["sending: MachinePair {machine0 = Machine {registers = fromList [('p',0),('x',3)], pc = 3, messageQueue = []}, machine1 = Machine {registers = fromList [('p',1)], pc = 0, messageQueue = [1,2,0]}}","sending: MachinePair {machine0 = Machine {registers = fromList [('a',1),('p',0),('x',3)], pc = 4, messageQueue = []}, machine1 = Machine {registers = fromList [('p',1),('x',1)], pc = 1, messageQueue = [1,2,0]}}","sending: MachinePair {machine0 = Machine {registers = fromList [('a',1),('b',2),('p',0),('x',3)], pc = 5, messageQueue = []}, machine1 = Machine {registers = fromList [('p',1),('x',2)], pc = 2, messageQueue = [1,2,0]}}"]),MachinePair {machine0 = Machine {registers = fromList [('a',1),('b',2),('c',1),('p',0),('x',3)], pc = 6, messageQueue = []}, machine1 = Machine {registers = fromList [('a',1),('b',2),('c',0),('p',1),('x',3)], pc = 6, messageQueue = []}})

In [30]:
runState (
    runReaderT (
        runWriterT executeInstructions
               ) 
        sampleInstructions
         ) 
         emptyMachinePair

(((),["sending: MachinePair {machine0 = Machine {registers = fromList [('a',0),('p',0),('x',1)], pc = 6, messageQueue = []}, machine1 = Machine {registers = fromList [('a',4),('p',1)], pc = 4, messageQueue = [4]}}"]),MachinePair {machine0 = Machine {registers = fromList [('a',4),('p',0),('x',1)], pc = 6, messageQueue = []}, machine1 = Machine {registers = fromList [('a',4),('p',1),('x',1)], pc = 6, messageQueue = []}})

In [31]:
sampleInstructions

[Set (Register 'a') (Literal 1),Add (Register 'a') (Literal 2),Mul (Register 'a') (Register 'a'),Mod (Register 'a') (Literal 5),Snd (Register 'a'),Set (Register 'a') (Literal 0),Rcv (Register 'a'),Jgz (Register 'a') (Literal (-1)),Set (Register 'a') (Literal 1),Jgz (Register 'a') (Literal (-2))]

In [32]:
part2 instructions = 
    runState (
        runReaderT (
            runWriterT executeInstructions
                   ) 
            instructions 
             ) 
             emptyMachinePair

In [35]:
main :: IO ()
main = do 
        text <- TIO.readFile "../../data/advent18.txt"
        let instrs = successfulParse text
        let ((result, l), statef) = part2 instrs
        print $ length l

In [36]:
main

5969

In [24]:
5969*2

11938

In [25]:
5969+6096

12065